In [4]:
import os
import numpy as np
import scipy as sp

In [5]:
class DataGenerator:
    """
    A class for reading and preprocessing text data.
    """

    def __init__(self, path: str, sequence_length: int):
        """
        Initializes a DataReader object with the path to a text file and the desired sequence length.

        Args:
            path (str): The path to the text file.
            sequence_length (int): The length of the sequences that will be fed to the self.
        """
        with open(path) as f:
            # Read the contents of the file
            self.data = f.read()

        # Find all unique characters in the text
        chars = list(set(self.data))

        # Create dictionaries to map characters to indices and vice versa
        self.char_to_idx = {ch: i for (i, ch) in enumerate(chars)}
        self.idx_to_char = {i: ch for (i, ch) in enumerate(chars)}

        # Store the size of the text data and the size of the vocabulary
        self.data_size = len(self.data)
        self.vocab_size = len(chars)

        # Initialize the pointer that will be used to generate sequences
        self.pointer = 0

        # Store the desired sequence length
        self.sequence_length = sequence_length


    def next_batch(self):
        """
        Generates a batch of input and target sequences.

        Returns:
            inputs_one_hot (np.ndarray): A numpy array with shape `(batch_size, vocab_size)` where each row is a one-hot encoded representation of a character in the input sequence.
            targets (list): A list of integers that correspond to the indices of the characters in the target sequence, which is the same as the input sequence shifted by one position to the right.
        """
        input_start = self.pointer
        input_end = self.pointer + self.sequence_length

        # Get the input sequence as a list of integers
        inputs = [self.char_to_idx[ch] for ch in self.data[input_start:input_end]]

        # One-hot encode the input sequence
        inputs_one_hot = np.zeros((len(inputs), self.vocab_size))
        inputs_one_hot[np.arange(len(inputs)), inputs] = 1

        # Get the target sequence as a list of integers
        targets = [self.char_to_idx[ch] for ch in self.data[input_start + 1:input_end + 1]]

        # Update the pointer
        self.pointer += self.sequence_length

        # Reset the pointer if the next batch would exceed the length of the text data
        if self.pointer + self.sequence_length + 1 >= self.data_size:
            self.pointer = 0

        return inputs_one_hot, targets

In [6]:
class GRU:
    """
    A class used to represent a Recurrent Neural Network (GRU).

    Attributes
    ----------
    hidden_size : int
        The number of hidden units in the GR.
    vocab_size : int
        The size of the vocabulary used by the GRU.
    sequence_length : int
        The length of the input sequences fed to the GRU.
    self.learning_rate : float
        The learning rate used during training.

    Methods
    -------
    __init__(hidden_size, vocab_size, sequence_length, self.learning_rate)
        Initializes an instance of the GRU class.
    """

    def __init__(self, hidden_size, vocab_size, sequence_length, learning_rate):
        """
        Initializes an instance of the GRU class.

        Parameters
        ----------
        hidden_size : int
            The number of hidden units in the GRU.
        vocab_size : int
            The size of the vocabulary used by the GRU.
        sequence_length : int
            The length of the input sequences fed to the GRU.
        learning_rate : float
            The learning rate used during training.
        """
        # hyper parameters
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.sequence_length = sequence_length
        self.learning_rate = learning_rate

        # model parameters
        self.Wz = np.random.uniform(-np.sqrt(1. / hidden_size), np.sqrt(1. / hidden_size),
                                    (hidden_size, hidden_size + vocab_size))
        self.bz = np.zeros((hidden_size, 1))

        self.Wr = np.random.uniform(-np.sqrt(1. / hidden_size), np.sqrt(1. / hidden_size),
                                    (hidden_size, hidden_size + vocab_size))
        self.br = np.zeros((hidden_size, 1))

        self.Wa = np.random.uniform(-np.sqrt(1. / hidden_size), np.sqrt(1. / hidden_size),
                                    (hidden_size, hidden_size + vocab_size))
        self.ba = np.zeros((hidden_size, 1))

        self.Wy = np.random.uniform(-np.sqrt(1. / hidden_size), np.sqrt(1. / hidden_size),
                                    (vocab_size, hidden_size))
        self.by = np.zeros((vocab_size, 1))

        # initialize gradients for each parameter
        self.dWz, self.dWr, self.dWa, self.dWy = np.zeros_like(self.Wz), np.zeros_like(self.Wr), np.zeros_like(
            self.Wa), np.zeros_like(self.Wy)
        self.dbz, self.dbr, self.dba, self.dby = np.zeros_like(self.bz), np.zeros_like(self.br), np.zeros_like(
            self.bz), np.zeros_like(self.by)

        # initialize parameters for adamw optimizer
        self.mWz = np.zeros_like(self.Wz)
        self.vWz = np.zeros_like(self.Wz)
        self.mWr = np.zeros_like(self.Wr)
        self.vWr = np.zeros_like(self.Wr)
        self.mWa = np.zeros_like(self.Wa)
        self.vWa = np.zeros_like(self.Wa)
        self.mWy = np.zeros_like(self.Wy)
        self.vWy = np.zeros_like(self.Wy)
        self.mbz = np.zeros_like(self.bz)
        self.vbz = np.zeros_like(self.bz)
        self.mbr = np.zeros_like(self.br)
        self.vbr = np.zeros_like(self.br)
        self.mba = np.zeros_like(self.ba)
        self.vba = np.zeros_like(self.ba)
        self.mby = np.zeros_like(self.by)
        self.vby = np.zeros_like(self.by)

    def sigmoid(self, x):
        """
        Computes the sigmoid activation function for a given input array.

        Parameters:
            x (ndarray): Input array.

        Returns:
            ndarray: Array of the same shape as `x`, containing the sigmoid activation values.
        """
        return 1 / (1 + np.exp(-x))

    def softmax(self, x):
        """
        Computes the softmax activation function for a given input array.

        Parameters:
            x (ndarray): Input array.

        Returns:
            ndarray: Array of the same shape as `x`, containing the softmax activation values.
        """
        # shift the input to prevent overflow when computing the exponentials
        x = x - np.max(x)
        # compute the exponentials of the shifted input
        p = np.exp(x)
        # normalize the exponentials by dividing by their sum
        return p / np.sum(p)

    def forward(self, X, c_prev, a_prev):
        """
        Performs forward propagation for a simple GRU model.

        Args:
            X (numpy array): Input sequence, shape (sequence_length, input_size)
            c_prev (numpy array): Previous cell state, shape (hidden_size, 1)
            a_prev (numpy array): Previous hidden state, shape (hidden_size, 1)

        Returns: X (numpy array): Input sequence, shape (sequence_length, input_size) c (dictionary): Cell state for
        each time step, keys = time step, values = numpy array shape (hidden_size, 1) r (dictionary): Reset gate for
        each time step, keys = time step, values = numpy array shape (hidden_size, 1) z (dictionary): Update gate for
        each time step, keys = time step, values = numpy array shape (hidden_size, 1) cc (dictionary): Candidate cell
        state for each time step, keys = time step, values = numpy array shape (hidden_size, 1) a (dictionary):
        Hidden state for each time step, keys = time step, values = numpy array shape (hidden_size, 1) y_pred (
        dictionary): Output probability vector for each time step, keys = time step, values = numpy array shape (
        output_size, 1)
        """

        # initialize dictionaries for backpropagation
        # initialize dictionaries for backpropagation
        r, z, c, cc, a, y_pred = {}, {}, {}, {}, {}, {}
        c[-1] = np.copy(c_prev)  # store the initial cell state in the dictionary
        a[-1] = np.copy(a_prev)  # store the initial hidden state in the dictionary

        # iterate over each time step in the input sequence
        for t in range(X.shape[0]):
            # concatenate the input and hidden state
            xt = X[t, :].reshape(-1, 1)
            concat = np.vstack((a[t - 1], xt))

            # compute the reset gate
            r[t] = self.sigmoid(np.dot(self.Wr, concat) + self.br)

            # compute the update gate
            z[t] = self.sigmoid(np.dot(self.Wz, concat) + self.bz)

            # compute the candidate cell state
            cc[t] = np.tanh(np.dot(self.Wa, np.vstack((r[t] * a[t - 1], xt))) + self.ba)

            # compute the cell state
            c[t] = z[t] * cc[t] + (1 - z[t]) * c[t - 1]

            # compute the hidden state
            a[t] = c[t]

            # compute the output probability vector
            y_pred[t] = self.softmax(np.dot(self.Wy, a[t]) + self.by)

        # return the output probability vectors, cell state, hidden state and gate vectors
        return X, r, z, c, cc, a, y_pred

    def backward(self, X, a_prev, c_prev, r, z, c, cc, a, y_pred, targets):
        """
        Performs backward propagation through time for a GRU network.

        Args:
            X (numpy array): Input sequence, shape (sequence_length, input_size)
            a_prev (numpy array): Previous hidden state, shape (hidden_size, 1)
            r (dictionary): Reset gate for each time step, keys = time step, values = numpy array shape (hidden_size, 1)
            z (dictionary): Update gate for each time step, keys = time step, values = numpy array shape (hidden_size, 1)
            c (dictionary): Cell state for each time step, keys = time step, values = numpy array shape (hidden_size, 1)
            cc (dictionary): Candidate cell state for each time step, keys = time step, values = numpy array shape (hidden_size, 1)
            a (dictionary): Hidden state for each time step, keys = time step, values = numpy array shape (hidden_size, 1)
            y_pred (dictionary): Output probability vector for each time step, keys = time step, values = numpy array shape (output_size, 1)
            targets (numpy array): Target outputs for each time step, shape (sequence_length, output_size)

        Returns:
            None       
        """
        # Initialize gradients for hidden state
        dc_next = np.zeros_like(c_prev)
        da_next = np.zeros_like(a_prev)

        # Iterate backwards through time steps
        for t in reversed(range(X.shape[0])):
            # compute the gradient of the output probability vector
            dy = np.copy(y_pred[t])
            dy[targets[t]] -= 1

            # compute the gradient of the output layer weights and biases
            self.dWy += np.dot(dy, a[t].T)
            self.dby += dy

            # compute the gradient of the hidden state
            da = np.dot(self.Wy.T, dy) + da_next

            # compute the gradient of the update gate
            xt = X[t, :].reshape(-1, 1)
            concat = np.vstack((a_prev, xt))
            dz = da * (a[t] - c[t])
            self.dWz += np.dot(dz, concat.T)
            self.dbz += dz

            # compute the gradient of the reset gate
            dr = da * np.dot(self.Wz[:, :self.hidden_size].T, dz) * (1 - r[t]) * r[t]
            self.dWr += np.dot(dr, concat.T)
            self.dbr += dr

            # compute the gradient of the current hidden state
            da = np.dot(self.Wa[:, :self.hidden_size].T, dr) + np.dot(self.Wz[:, :self.hidden_size].T, dz)
            self.dWa += np.dot(da * (1 - a[t]**2), concat.T)
            self.dba += da * (1 - a[t]**2)

            # compute the gradient of the input to the next hidden state
            da_next = np.dot(self.Wr[:, :self.hidden_size].T, dr) \
                      + np.dot(self.Wz[:, :self.hidden_size].T, dz) \
                      + np.dot(self.Wa[:, :self.hidden_size].T, da)
        # clip gradients to avoid exploding gradients
        for grad in [self.dWz, self.dWr, self.dWa, self.dWy, self.dbz, self.dbr, self.dba, self.dby]:
            np.clip(grad, -1, 1)

    def loss(self, y_preds, targets):
        """
        Computes the cross-entropy loss for a given sequence of predicted probabilities and true targets.

        Parameters:
            y_preds (ndarray): Array of shape (sequence_length, vocab_size) containing the predicted probabilities for each time step.
            targets (ndarray): Array of shape (sequence_length, 1) containing the true targets for each time step.

        Returns:
            float: Cross-entropy loss.
        """
        # calculate cross-entropy loss
        return sum(-np.log(y_preds[t][targets[t], 0]) for t in range(self.sequence_length))

    def adamw(self, beta1=0.9, beta2=0.999, epsilon=1e-8, L2_reg=1e-4):
        """
        Updates the GRU's parameters using the AdamW optimization algorithm.
        """
        
        # AdamW update for Wz
        self.mWz = beta1 * self.mWz + (1 - beta1) * self.dWz
        self.vWz = beta2 * self.vWz + (1 - beta2) * np.square(self.dWz)
        m_hat = self.mWz / (1 - beta1)
        v_hat = self.vWz / (1 - beta2)
        self.Wz -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.Wz)

        # AdamW update for bu
        self.mbz = beta1 * self.mbz + (1 - beta1) * self.dbz
        self.vbz = beta2 * self.vbz + (1 - beta2) * np.square(self.dbz)
        m_hat = self.mbz / (1 - beta1)
        v_hat = self.vbz / (1 - beta2)
        self.bz -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.bz)

        # AdamW update for Wr
        self.mWr = beta1 * self.mWr + (1 - beta1) * self.dWr
        self.vWr = beta2 * self.vWr + (1 - beta2) * np.square(self.dWr)
        m_hat = self.mWr / (1 - beta1)
        v_hat = self.vWr / (1 - beta2)
        self.Wr -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.Wr)

        # AdamW update for br
        self.mbr = beta1 * self.mbr + (1 - beta1) * self.dbr
        self.vbr = beta2 * self.vbr + (1 - beta2) * np.square(self.dbr)
        m_hat = self.mbr / (1 - beta1)
        v_hat = self.vbr / (1 - beta2)
        self.br -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.br)

        # AdamW update for Wa
        self.mWa = beta1 * self.mWa + (1 - beta1) * self.dWa
        self.vWa = beta2 * self.vWa + (1 - beta2) * np.square(self.dWa)
        m_hat = self.mWa / (1 - beta1)
        v_hat = self.vWa / (1 - beta2)
        self.Wa -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.Wa)

        # AdamW update for br
        self.mba = beta1 * self.mba + (1 - beta1) * self.dba
        self.vba = beta2 * self.vba + (1 - beta2) * np.square(self.dba)
        m_hat = self.mba / (1 - beta1)
        v_hat = self.vba / (1 - beta2)
        self.ba -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.ba)

        # AdamW update for Wy
        self.mWy = beta1 * self.mWy + (1 - beta1) * self.dWy
        self.vWy = beta2 * self.vWy + (1 - beta2) * np.square(self.dWy)
        m_hat = self.mWy / (1 - beta1)
        v_hat = self.vWy / (1 - beta2)
        self.Wy -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.Wy)

        # AdamW update for by
        self.mby = beta1 * self.mby + (1 - beta1) * self.dby
        self.vby = beta2 * self.vby + (1 - beta2) * np.square(self.dby)
        m_hat = self.mby / (1 - beta1)
        v_hat = self.vby / (1 - beta2)
        self.by -= self.learning_rate * (m_hat / (np.sqrt(v_hat) + epsilon) + L2_reg * self.by)

    def train(self, data_generator,iterations):
        """
        Train the GRU on a dataset using backpropagation through time.

        Args:
            data_generator: An instance of DataGenerator containing the training data.

        Returns:
            None
        """
        iter_num = 0
        # stopping criterion for training
        threshold = 50
    
        smooth_loss = -np.log(1.0 / data_generator.vocab_size) * self.sequence_length  # initialize loss
        while (iter_num < iterations):
            # initialize hidden state at the beginning of each sequence
            if data_generator.pointer == 0:
                c_prev = np.zeros((self.hidden_size, 1))
                a_prev = np.zeros((self.hidden_size, 1))

            # get a batch of inputs and targets
            inputs, targets = data_generator.next_batch()

            # forward pass
            X, r, z, c, cc, a, y_pred = self.forward(inputs, c_prev, a_prev)

            # backward pass
            self.backward(X, a_prev, c_prev, r, z, c, cc, a, y_pred, targets)

            # calculate and update loss
            loss = self.loss(y_pred, targets)
            self.adamw()
            smooth_loss = smooth_loss * 0.999 + loss * 0.001
            # update previous hidden state for the next batch
            a_prev = a[self.sequence_length - 1]
            c_prev = c[self.sequence_length - 1]
#             if iter_num == 5900 or iter_num == 30000:
#                         self.learning_rate *= 0.1
            # print progress every 100 iterations
            if iter_num % 100 == 0:
#                 self.learning_rate *= 0.99
                sample_idx = self.sample(c_prev, a_prev, inputs[0, :], 200)
                print(''.join(data_generator.idx_to_char[idx] for idx in sample_idx))
                print("\n\niter :%d, loss:%f" % (iter_num, smooth_loss))
            iter_num += 1

    def sample(self, c_prev, a_prev, seed_idx, n):
        """
        Sample a sequence of integers from the model.

        Args:
            c_prev (numpy.ndarray): Previous cell state, a numpy array of shape (hidden_size, 1).
            a_prev (numpy.ndarray): Previous hidden state, a numpy array of shape (hidden_size, 1).
            seed_idx (numpy.ndarray): Seed letter from the first time step, a numpy array of shape (vocab_size, 1).
            n (int): Number of characters to generate.

        Returns:
            list: A list of integers representing the generated sequence.

        """
        # initialize input and seed_idx
        x = np.zeros((self.vocab_size, 1))
        # convert one-hot encoding to integer index
        seed_idx = np.argmax(seed_idx, axis=-1)

        # set the seed letter as the input for the first time step
        x[seed_idx] = 1

        # generate sequence of characters
        idxes = []
        c = np.copy(c_prev)
        a = np.copy(a_prev)
        for t in range(n):
            # compute the hidden state and cell state
            concat = np.vstack((a, x))
            z = self.sigmoid(np.dot(self.Wz, concat) + self.bz)
            r = self.sigmoid(np.dot(self.Wr, concat) + self.br)
            cc = np.tanh(np.dot(self.Wa, np.vstack((r * a, x))) + self.ba)
            c = z * c + (1 - z) * cc
            a = c
            # compute the output probabilities
            y = self.softmax(np.dot(self.Wy, a) + self.by)

            # sample the next character from the output probabilities
            idx = np.random.choice(range(self.vocab_size), p=y.ravel())

            # set the input for the next time step
            x = np.zeros((self.vocab_size, 1))
            x[idx] = 1

            # append the sampled character to the sequence
            idxes.append(idx)

        # return the generated sequence
        return idxes

    def predict(self, data_generator, start, n):
        """
        Generate a sequence of n characters using the trained GRU model, starting from the given start sequence.

        Args:
        - data_generator: an instance of DataGenerator
        - start: a string containing the start sequence
        - n: an integer indicating the length of the generated sequence

        Returns:
        - txt: a string containing the generated sequence
        """
        # initialize input sequence
        x = np.zeros((self.vocab_size, 1))
        chars = [ch for ch in start]
        idxes = []
        for i in range(len(chars)):
            idx = data_generator.char_to_idx[chars[i]]
            x[idx] = 1
            idxes.append(idx)
        # initialize cell state and hidden state
        a = np.zeros((self.hidden_size, 1))
        c = np.zeros((self.hidden_size, 1))

        # generate new sequence of characters
        for t in range(n):
            # compute the hidden state and cell state
            concat = np.vstack((a, x))

            # compute the reset gate
            r = self.sigmoid(np.dot(self.Wr, concat) + self.br)

            # compute the update gate
            z = self.sigmoid(np.dot(self.Wz, concat) + self.bz)

            # compute the candidate cell state
            cc = np.tanh(np.dot(self.Wa, np.vstack((r * a, x))) + self.ba)

            # compute the cell state
            c = z * cc + (1 - z) * c

            # compute the hidden state
            a = c

            # compute the output probability vector
            y_pred = self.softmax(np.dot(self.Wy, a) + self.by)
            # sample the next character from the output probabilities
            idx = np.random.choice(range(self.vocab_size), p=y_pred.ravel())
            x = np.zeros((self.vocab_size, 1))
            x[idx] = 1
            idxes.append(idx)
        txt = ''.join(data_generator.idx_to_char[i] for i in idxes)
        return txt

In [7]:
sequence_length = 24
#read text from the "input.txt" file
data_generator = DataGenerator('text.txt', sequence_length)
gru =  GRU(hidden_size=100, vocab_size=data_generator.vocab_size,sequence_length=sequence_length,learning_rate=0.005)

gru.train(data_generator,iterations=6000)

H,U..zjLWI3j-UR'nlGfkjB&hhPJPTp&U'qRLbeV?$fIs&PZ:uHRQQNWALLJDa;b&RmNX.lq!;;KxuBgXoQGZcFNVa.nnaFeXIHWUmMxq'&H
B!YPfhHC:s?rn;:aZMQya
zplJj,3SvgTn;LnjXDqVaTdXy
JIwKJcdLbQxhXHkW&WPh'xGDlnSn?vJ:RHCvOYZqjaO


iter :0, loss:100.185286
g aes ttyntienaityutiledHhre ro  titsuteeol
e cwH
koitiihtnhgBt aim,k;rtyc
VrWct
o we t
o neo heynss rar aehltiene nodrsa uoen t
it?
i.nrhs foueakkjoce
 e aaike
yib  ei r htth
Ihi ois inoontaenetdhtic


iter :100, loss:98.257050
 us,
e atthgce a ar tothYYfie h ay wleei thsayeeerofdop ous ns tatndtoui fhntie te a ora i col ne eore c eore  
 we r o ninit oenng ay tollyvu egtl.s
ieer otue wOsaoudan
ee waoum aaaemdis nt muss fyou


iter :200, loss:95.729453
hre tg hat CugY$gd aene  wsye  far eeoeris tthor,l s,thi ep:

t hey te,hel b wthe dir c,aner,te iti e mrhe, b mire  b,sSl hest ttheranddw pbyirourm wbyousllll.ius,henanly comree th benddi o bsMl e ae 


iter :300, loss:93.149231
indae bto mdingt halellnell yo wogihennde ali ihdotbo
ey
lyl ofouhe  os
 ouo ufori ns

In [8]:
gru.predict(data_generator, "c", 1000)

"cerus thas that\nThas bru, May pricsit reitne, band wirs pearvife he wal be lour nusto thos nhely;\nTht biked erur,\n\nTh hh ppont he lalds bun dearve nuf re fay onurary sra lde my, o do dt Sererefrurgans, 'ld\nThery.\n\nSCtir,\nAy silko, mo bad our I hier nut\nty\nORUMINIl Ad sa'l,\nButur, is ugssarc kouf fe fledr the wror fur sthe myo upresty,\nHapirt shal; I pre dmel n:\nYourperins turga, bat hi for ldu hpavi bay urver sory uplaprer fore ne yob urill kellsy uits nwor furs we waod otusr, ray urt tho he goprat lispats teld; wht hai pres rikus!\n\nOMaNI In uld waerus, tur bos tol my eful dos anst harad lass the ne by lass 'tr!\n\nThthit\nThals moun kens by ourmpelli\nAnis bfol the dy rary auppre nfare saky prpery.\nd cou nd Secourkou pns thapte, d wear pat hipis flais s purave styom.\n\nSI for bles wod han fo brald fat enye owlld eve.\n\nThes fresal I whil fur haly sior ury wad athor whallve wo uels pury, wikisld naknes serend erbeny\nA uppraknt he fur woll fe reve\nrurnd ais bey yoru